# Airbyte Hubspot | Airbyte HubSpot



> [Airbyte](https://github.com/airbytehq/airbyte)は、API、データベース、ファイルからデータウェアハウスやデータレイクへのELTパイプラインのためのデータ統合プラットフォームです。データウェアハウスとデータベースへのELTコネクターのカタログが最も豊富です。
>
> > [Airbyte](https://github.com/airbytehq/airbyte) is a data integration platform for ELT pipelines from APIs, databases & files to warehouses & lakes. It has the largest catalog of ELT connectors to data warehouses and databases.

このローダーは、Hubspotコネクタをドキュメントローダーとして公開し、様々なHubspotオブジェクトをドキュメントとして読み込むことを可能にします。

> This loader exposes the Hubspot connector as a document loader, allowing you to load various Hubspot objects as documents.



## Installation | インストール



まず、`airbyte-source-hubspot` Pythonパッケージをインストールする必要があります。

> First, you need to install the `airbyte-source-hubspot` python package.



In [ ]:
#!pip install airbyte-source-hubspot

## Example | 例



リーダーの設定方法の詳細については、[Airbyteのドキュメントページ](https://docs.airbyte.com/integrations/sources/hubspot/)をご覧ください。設定オブジェクトが準拠すべきJSONスキーマはGitHubで見つけることができます: <https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-hubspot/source_hubspot/spec.yaml>。

> Check out the [Airbyte documentation page](https://docs.airbyte.com/integrations/sources/hubspot/) for details about how to configure the reader.
> The JSON schema the config object should adhere to can be found on Github: <https://github.com/airbytehq/airbyte/blob/master/airbyte-integrations/connectors/source-hubspot/source_hubspot/spec.yaml>.

一般的な形は次のようになります：

> The general shape looks like this:

```python
{
  "start_date": "<date from which to start retrieving records from in ISO format, e.g. 2020-10-20T00:00:00Z>",
  "credentials": {
    "credentials_title": "Private App Credentials",
    "access_token": "<access token of your private app>"
  }
}
```

デフォルトでは、すべてのフィールドはドキュメントのメタデータとして保存され、テキストは空文字列に設定されています。リーダーから返されたドキュメントを変換して、ドキュメントのテキストを構築してください。

> By default all fields are stored as metadata in the documents and the text is set to an empty string. Construct the text of the document by transforming the documents returned by the reader.



In [ ]:
from langchain.document_loaders.airbyte import AirbyteHubspotLoader

config = {
    # your hubspot configuration
}

loader = AirbyteHubspotLoader(
    config=config, stream_name="products"
)  # check the documentation linked above for a list of all streams

これで、通常の方法でドキュメントを読み込むことができます

> Now you can load documents the usual way



In [ ]:
docs = loader.load()

`load`はリストを返すため、すべてのドキュメントが読み込まれるまで処理がブロックされます。このプロセスをより良くコントロールするために、イテレータを返す`lazy_load`メソッドを使用することもできます：

> As `load` returns a list, it will block until all documents are loaded. To have better control over this process, you can also you the `lazy_load` method which returns an iterator instead:



In [ ]:
docs_iterator = loader.lazy_load()

デフォルトではページの内容は空であり、メタデータオブジェクトにはレコードからのすべての情報が含まれていることに注意してください。ドキュメントを処理するためには、ベースローダーを継承するクラスを作成し、`_handle_records` メソッドを自分で実装する必要があります：

> Keep in mind that by default the page content is empty and the metadata object contains all the information from the record. To process documents, create a class inheriting from the base loader and implement the `_handle_records` method yourself:



In [ ]:
from langchain.docstore.document import Document


def handle_record(record, id):
    return Document(page_content=record.data["title"], metadata=record.data)


loader = AirbyteHubspotLoader(
    config=config, record_handler=handle_record, stream_name="products"
)
docs = loader.load()

## Incremental loads | インクリメンタルロード

一部のストリームはインクリメンタルローディングを許可しており、これはソースが同期されたレコードを追跡し、それらを再度ロードしないことを意味します。これは、データ量が多く頻繁に更新されるソースにとって有用です。

> Some streams allow incremental loading, this means the source keeps track of synced records and won't load them again. This is useful for sources that have a high volume of data and are updated frequently.

これを利用するためには、ローダーの `last_state` プロパティを保存しておき、ローダーを再度作成する際にそれを渡してください。これにより、新しいレコードのみがロードされることが保証されます。

> To take advantage of this, store the `last_state` property of the loader and pass it in when creating the loader again. This will ensure that only new records are loaded.



In [ ]:
last_state = loader.last_state  # store safely

incremental_loader = AirbyteHubspotLoader(
    config=config, stream_name="products", state=last_state
)

new_docs = incremental_loader.load()